# The relationship between US interest rates and public debt in developing countries

## Setup of the Workspace

### Packages

In [20]:
import pandas as pd
import numpy as np

import os 

#from sqlalchemy import create_engine

### Paths

In [3]:
ROOT = os.path.dirname(os.getcwd())

PATH_DATA = os.path.join(ROOT, 'data')


## Data

### FRED
contains data on US interest rates

In [4]:
# Download and read the CSV directly into a DataFrame
#df_fred = pd.read_csv("C:/Users/Maude/OneDrive/Documents/etudes/UZH/PhD/2nd year/first semester/programming/project/data/FEDFUNDS.csv")

df_fred_raw = pd.read_csv(os.path.join(PATH_DATA, 'FEDFUNDS.csv')) #.replace('\\', '/'))



#convert monthly data to yearly data

## Convert the date column to datetime format
df_fred_raw['DATE'] = pd.to_datetime(df_fred_raw['DATE'])

## Filter the DataFrame to include only rows where the month is January
df_fred_raw_year = df_fred_raw[df_fred_raw['DATE'].dt.month == 1]

## Extract and keep only the year
df_fred_raw_year['DATE'] = df_fred_raw_year['DATE'].dt.year



#convert data to same format as WB to append

## Add columns for 'Country Name' and 'Country Code'
df_fred_raw_year['Country Name'] = 'United States'

df_fred_raw_year['Country Code'] = 'USA'

## Rename the columns
df_fred_raw_year.rename(columns={'DATE': 'YEAR', 'FEDFUNDS': 'Series Name'}, inplace=True)

df_fred_raw_year.head(10)

## Convert the DataFrame from long format to wide format
df_fred_raw_wide = df_fred_raw_year.pivot(index=['Country Name', 'Country Code'], columns='YEAR', values='Series Name')



## Add a new column 'Series name' with the value 'Interest rate'
df_fred_raw_wide['Series Name'] = 'Interest rate'
df_fred_raw_wide['Country Name'] = 'United States'
df_fred_raw_wide['Country Code'] = 'USA'

## Drop the columns 1955, 1956, 1957, 1958, 1959, 2024
df_fred_raw_wide.drop(columns=[1955, 1956, 1957, 1958, 1959, 2024], inplace=True)

df_fred_raw_wide.head(10)




C:\Users\Maude\AppData\Local\Temp\ipykernel_8460\239376244.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fred_raw_year['DATE'] = df_fred_raw_year['DATE'].dt.year
C:\Users\Maude\AppData\Local\Temp\ipykernel_8460\239376244.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fred_raw_year['Country Name'] = 'United States'
C:\Users\Maude\AppData\Local\Temp\ipykernel_8460\239376244.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

,YEAR,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,...,2017,2018,2019,2020,2021,2022,2023,Series Name,Country Name,Country Code
Country Name,Country Code,,,,,,,,,,,,,,,,,,,,,
United States,USA,3.99,1.45,2.15,2.92,3.48,3.9,4.42,4.94,4.61,6.3,...,0.65,1.41,2.4,1.55,0.09,0.08,4.33,Interest rate,United States,USA


### World Bank


contains data on GNI, GNI per capita, population, external debt stock (% of GNI), external debt stock, official exchange rate, currency composition of PPG

In [6]:
df_WB_raw = pd.read_csv(os.path.join(PATH_DATA, 'World_Bank_data.csv')) #.replace('\\', '/'))
df_WB_raw.head()
#df_WB_raw.shape


,Country Name,Country Code,Series Name,Series Code,1960 [YR1960],1961 [YR1961],1962 [YR1962],1963 [YR1963],1964 [YR1964],1965 [YR1965],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,External debt stocks (% of GNI),DT.DOD.DECT.GN.ZS,..,..,..,..,..,..,...,12.3049882430693,13.4908300391384,14.1470450328557,14.5133427332625,14.6424246274665,13.9496520911751,15.1208585234171,24.7744403222298,..,..
1,Afghanistan,AFG,"External debt stocks, total (DOD, current US$)",DT.DOD.DECT.CD,..,..,..,..,..,..,...,2529865267.8,2596917266.2,2596050086.4,2751986638.9,2678760424.2,2661685956.5,3040072312.4,3555784339.6,3393247241.7,..
2,Afghanistan,AFG,"Official exchange rate (LCU per US$, period av...",PA.NUS.FCRF,17.1965606878624,17.1965606878624,17.1965606878624,35.1096447377192,38.6922615476905,38.6922615476905,...,57.2475,61.1434615416667,67.8660857692308,68.0269040822312,72.083247177304,77.7379491783367,76.8135364354897,..,..,..
3,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,548888848.888889,560000022.222222,557777806.666667,766666706.666667,815555562.222222,1026666677.77778,...,20559672324.9094,19249499438.7455,18350475875.2999,18961769787.4548,18294513957.5523,19080662010.7479,20105156769.1907,14352632363.74,14545219074.5763,..
4,Afghanistan,AFG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,..,..,..,..,..,..,...,2240,2300,2250,2360,2470,2630,2590,2150,2100,..


In [7]:
df_WB_comp_raw = pd.read_csv(os.path.join(PATH_DATA, 'World_Bank_composition.csv')) #.replace('\\', '/'))
df_WB_comp_raw.shape
df_WB_comp_raw.head()

,Country Name,Country Code,Counterpart-Area Name,Counterpart-Area Code,Series Name,Series Code,1970 [YR1970],1971 [YR1971],1972 [YR1972],1973 [YR1973],...,2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022],2023 [YR2023]
0,Afghanistan,AFG,World,WLD,"Currency composition of PPG debt, U.S. dollars...",DT.CUR.USDL.ZS,..,..,..,..,...,72.119,71.5365,71.6963,71.4215,71.4005,70.7864,70.6726,71.0905,71.5724,..
1,Albania,ALB,World,WLD,"Currency composition of PPG debt, U.S. dollars...",DT.CUR.USDL.ZS,..,..,..,..,...,37.0724,33.7267,34.6949,38.6747,39.4211,41.9899,37.2818,34.3902,35.4623,..
2,Algeria,DZA,World,WLD,"Currency composition of PPG debt, U.S. dollars...",DT.CUR.USDL.ZS,36.9648,41.1115,43.7869,39.446,...,12.1137,10.7537,3.4616,1.2213,0.0891,0.0084,0.0000,0.0000,0.0000,..
3,Angola,AGO,World,WLD,"Currency composition of PPG debt, U.S. dollars...",DT.CUR.USDL.ZS,..,..,..,..,...,90.8066,91.2840,91.8073,91.0572,91.7833,91.6605,90.7719,90.4606,90.5126,..
4,Argentina,ARG,World,WLD,"Currency composition of PPG debt, U.S. dollars...",DT.CUR.USDL.ZS,45.6812,49.7009,52.5138,47.255,...,69.7062,69.6138,66.6773,67.5774,71.6776,73.6521,89.3016,90.7706,91.4307,..


In [8]:
### Appended dataframes

# Append WB dataframes
WB_df_raw = pd.concat([df_WB_raw, df_WB_comp_raw], ignore_index=True)

# Delete columns Counterpart-Area Code and Counterpart-Area Name

WB_df_raw_del = WB_df_raw.drop(columns=['Counterpart-Area Code', 'Counterpart-Area Name'])

# Rename years columns

## Create a dictionary to map old column names to new column names
rename_dict = {col: col.split('[')[0] for col in WB_df_raw_del.columns if '[' in col}

## Rename the columns
WB_df_raw_del.rename(columns=rename_dict, inplace=True)

# Create a dictionary to map old column names to new column names
rename_dict = {f'{year} ': year for year in range(1960, 2024)}

# Rename the columns
WB_df_raw_del.rename(columns=rename_dict, inplace=True)

WB_df_raw_del.head()


,Country Name,Country Code,Series Name,Series Code,1960,1961,1962,1963,1964,1965,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,AFG,External debt stocks (% of GNI),DT.DOD.DECT.GN.ZS,..,..,..,..,..,..,...,12.3049882430693,13.4908300391384,14.1470450328557,14.5133427332625,14.6424246274665,13.9496520911751,15.1208585234171,24.7744403222298,..,..
1,Afghanistan,AFG,"External debt stocks, total (DOD, current US$)",DT.DOD.DECT.CD,..,..,..,..,..,..,...,2529865267.8,2596917266.2,2596050086.4,2751986638.9,2678760424.2,2661685956.5,3040072312.4,3555784339.6,3393247241.7,..
2,Afghanistan,AFG,"Official exchange rate (LCU per US$, period av...",PA.NUS.FCRF,17.1965606878624,17.1965606878624,17.1965606878624,35.1096447377192,38.6922615476905,38.6922615476905,...,57.2475,61.1434615416667,67.8660857692308,68.0269040822312,72.083247177304,77.7379491783367,76.8135364354897,..,..,..
3,Afghanistan,AFG,GNI (current US$),NY.GNP.MKTP.CD,548888848.888889,560000022.222222,557777806.666667,766666706.666667,815555562.222222,1026666677.77778,...,20559672324.9094,19249499438.7455,18350475875.2999,18961769787.4548,18294513957.5523,19080662010.7479,20105156769.1907,14352632363.74,14545219074.5763,..
4,Afghanistan,AFG,"GNI per capita, PPP (current international $)",NY.GNP.PCAP.PP.CD,..,..,..,..,..,..,...,2240,2300,2250,2360,2470,2630,2590,2150,2100,..


### Append FRED and WB 

In [22]:
data_FRED_WB_wide = pd.concat([WB_df_raw_del, df_fred_raw_wide], ignore_index=True)

# Replace values equal to '..' with NaN
data_FRED_WB_wide.replace('..', np.nan, inplace=True)

# Drop rows where the "Country Code" column contains NaN
data_FRED_WB_wide.dropna(subset=['Country Code'], inplace=True)

# Drop column Series Code
data_FRED_WB_wide.drop(columns='Series Code', inplace=True)

# Convert the DataFrame from wide format to long format
data_FRED_WB_long = pd.melt(data_FRED_WB_wide, id_vars=['Country Name', 'Country Code', 'Series Name'], var_name='Year', value_name='Value')



data_FRED_WB_wide.head(10)  
#data_FRED_WB_long.tail(10)


,Country Name,Country Code,Series Name,1960,1961,1962,1963,1964,1965,1966,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Afghanistan,AFG,External debt stocks (% of GNI),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,12.3049882430693,13.4908300391384,14.1470450328557,14.5133427332625,14.6424246274665,13.9496520911751,15.1208585234171,24.7744403222298,NaN,NaN
1,Afghanistan,AFG,"External debt stocks, total (DOD, current US$)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2529865267.8,2596917266.2,2596050086.4,2751986638.9,2678760424.2,2661685956.5,3040072312.4,3555784339.6,3393247241.7,NaN
2,Afghanistan,AFG,"Official exchange rate (LCU per US$, period av...",17.1965606878624,17.1965606878624,17.1965606878624,35.1096447377192,38.6922615476905,38.6922615476905,38.6922615476905,...,57.2475,61.1434615416667,67.8660857692308,68.0269040822312,72.083247177304,77.7379491783367,76.8135364354897,NaN,NaN,NaN
3,Afghanistan,AFG,GNI (current US$),548888848.888889,560000022.222222,557777806.666667,766666706.666667,815555562.222222,1026666677.77778,1428888871.11111,...,20559672324.9094,19249499438.7455,18350475875.2999,18961769787.4548,18294513957.5523,19080662010.7479,20105156769.1907,14352632363.74,14545219074.5763,NaN
4,Afghanistan,AFG,"GNI per capita, PPP (current international $)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2240,2300,2250,2360,2470,2630,2590,2150,2100,NaN
5,Afghanistan,AFG,"Population, total",8622466,8790140,8969047,9157465,9355514,9565147,9783147,...,32716210,33753499,34636207,35643418,36686784,37769499,38972230,40099462,41128771,42239854
6,Albania,ALB,External debt stocks (% of GNI),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,63.7664595243345,73.3135322383179,70.651576129568,75.0981975562841,63.7541024238069,61.015763100267,70.2589295385165,62.5241703121563,56.3023228650404,NaN
7,Albania,ALB,"External debt stocks, total (DOD, current US$)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8512452310,8447046910.5,8516222980.3,9801454504.1,9651086366.7,9274545228.5,10477366100.7,11054877741.5,10454989641.6,NaN
8,Albania,ALB,"Official exchange rate (LCU per US$, period av...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,105.48,125.961666666667,124.1425,119.1,107.989166666667,109.850833333333,108.65,103.52,113.041666666667,100.645
9,Albania,ALB,GNI (current US$),NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13349419700.4171,11521811393.6203,12053832974.203,13051517643.6213,15137984850.8323,15200244588.0406,14912504601.9328,17680966714.6445,18569375310.9639,22674870567.8374


### Classification of low income countries

In [10]:
low_inc_2024 = ["Afghanistan", "Burkina Faso", "Burundi", "Central African Republic", "Chad", "Congo, Dem. Rep.", "Eritrea", "Ethiopia", "Gambia, The", "Guinea-Bissau", "Korea, Dem. People’s Rep.", "Liberia", "Madagascar", "Malawi", "Mali", "Mozambique", "Niger", "Rwanda", "Sierra Leone", "Somalia", "South Sudan", "Sudan", "Syrian Arab Republic", "Togo", "Uganda" "Yemen, Rep."]
